We will define our genetic algorithm mapper with the following algorithm. For our algorithm, we use the following definition of fitness.
fitness = 1 / (latency  energy)
n = 5, k = 20, p = 10

1. Generate n = 5 randomly ordered strings of the valid dataflow, that is a random permutation of [R, S, P, Q, C, M, N]. Initialize f to 0.
2. Initialize a goal fitness g, dependent on latency and energy.
3. While f>g,
    Mutation: For i from 1 to n mutations, mutate each permutation k/n times to get k mutations. For each mutation, pick two parameters at random and swap them.
    Selection: Calculate latency and energy and evaluate the fitness of each k mutations. Take the p = 10 with the highest fitness.
    Crossover: Take pairs of p = 10 mutations and crossover to get n = 5 permutations. Let f = top fitness from these permutations.
4. Return best permutation.

In [34]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from loaders import *
import yaml
from yaml import load

In [35]:
show_config('designs/system/constraints.yaml')

constraints:
  version: 0.4
  targets:
  - target: DRAM
    type: dataspace
    keep: [Inputs, Outputs, Weights]
    bypass: []
  - target: DRAM
    type: temporal
    permutation: [R, S, Q, P, C, M, N]
  - target: global_buffer
    type: dataspace
    keep: [Inputs, Outputs, Weights]
    bypass: []
  - target: global_buffer
    type: temporal
    permutation: [S, R, Q, P, C, M, N]
  - target: PE
    type: spatial
    permutation: [C, M, R, S, P, Q, N]
    split: 1
  - target: scratchpad
    type: dataspace
    keep: [Weights]
    bypass: [Inputs, Outputs]
  - target: scratchpad
    type: temporal
    permutation: [Q, P, N, C, M, S, R]

  - target: weight_reg
    type: dataspace
    keep: [Weights]
    bypass: [Inputs, Outputs]
  - target: weight_reg
    type: temporal
    permutation: [R, S, P, Q, C, M, N]
  - target: input_activation_reg
    type: dataspace
    keep: [Inputs]
    bypass: [Weights, Outputs]
  - target: input_activation_reg
    type: temporal
    permutation: [R, S, P,

In [42]:
def dummy_fitness(dataflow):
    return random.uniform(0, 1.0e-7)   

In [57]:
class GeneticAlgorithm:
    def __init__(
            self, 
            dataflow=['R', 'S', 'P', 'Q', 'C', 'M', 'N'], 
            workload='layer_shapes/fc1.yaml', 
            pe_dims={'pe_meshX': 2, 'pe_meshY': 8}, 
            mapper='designs/_include/mapper.yaml',
            constraints='designs/system/constraints.yaml',
            architecture='designs/system/arch.yaml',
            n=5, 
            k=20, 
            p=10, 
            iter=10,
            ):
        # convolution
        self.dataflow = dataflow
        # workload = 'layer_shapes/conv2.yaml'
        # workload = 'layer_shapes/conv1.yaml'
        self.workload = workload
        # pe_dims = {'pe_meshX': 1, 'pe_meshY': 16}
        self.pe_dims = pe_dims
        # pe_dims = {'pe_meshX': 4, 'pe_meshY': 4}

        # n population -> k mutations -> p selection -> n population
        # constraints: n | k, p/2 = n
        self.mapper=mapper
        self.constraints=constraints
        self.architecture=architecture
        self.n = n
        self.k = k
        self.p = p
        self.iter = iter
        self.mapper_call_count = 0
        self.VISITED={}
        self.mapper_calls = [] # list of [permutation, fitness] in order of call


    def fitness(self, dataflow):
        '''
        Calculates the fitness of a given permustation `dataflow`. Returns the
        inverse EDP after running the timeloop mapper on this dataflow.
        '''
        data = self.evaluate(dataflow)
        energy, latency = data
        inverse_EDP = 1 / (energy * latency)
        print(f"{dataflow} has a fitness of {inverse_EDP}")

        # update visited
        self.VISITED[''.join(dataflow)] = inverse_EDP
        self.mapper_calls.append([''.join(dataflow), inverse_EDP])
        return inverse_EDP


    def evaluate(self, dataflow):
        '''
        Evaluates the given dataflow on -- architecture

        dataflow: computation ordering in list format
        workload: the file path to the workload this is being evaluated on
        returns tuple of energy, latency
        '''

        # create a new constraints file with the new PE permutation
        stream = open(self.constraints, 'r')
        dictionary = yaml.safe_load(stream)
        # print(dictionary['constraints']['targets'][4])
        idx = 4 # PE
        dictionary['constraints']['targets'][idx]['permutation'] = dataflow

        filename = ''.join(dataflow)
        with open(f'iters/configs/{filename}.yaml', 'w') as file:
            yaml.dump(dictionary, file, default_flow_style=False)

        constraints = f'iters/configs/{filename}.yaml'

        sys_1x16_result = run_timeloop_mapper( # TODO: this should be run_timeloop_mapper not run_timeloop_model!
            # config,
            self.pe_dims,
            architecture=self.architecture,
            mapper=self.mapper,
            problem=self.workload,
            constraints=constraints 
        )

        # keeping track of mapper call count for data purposes
        self.mapper_call_count += 1
        
        stats = open('./output_dir/timeloop-mapper.stats.txt', 'r').read()
        mapping = sys_1x16_result.mapping

        lines = stats.split('\n')
        energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
        cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
        # min_energy = min(min_energy, energy)

        print(energy, cycles)
        return energy, cycles


    def mutation(self, population):
        """
        Performs a random swap mutation for every member of `population`.
        Returns a list of the new mutated population.
        """
        mutations = []

        for df in population:
            for _ in range(self.k // self.n):
                mutation = df.copy()
                # swap two random indices
                idx1, idx2 = random.sample(range(len(self.dataflow)), 2)
                mutation[idx1], mutation[idx2] = mutation[idx2], mutation[idx1]
                mutations.append(mutation)
        return mutations


    def selection(self, population: list[str], p: int, fitness, workload: str, pe_dims) -> list:
        """
        Evaluates the fitness using `fitness` func of all members of a population on
        `workload` workload with pe dimensions `pe_dims`, and performs selection of 
        the top p candidates.
        """
        print("selection")
        fitnesses = []
        i = 0
        for candidate in population:
            i += 1
            print(f'{i}/{len(population)} candidate')
            # do a check to see if we've already run this config
            trial_name = ''.join(candidate)
            if trial_name in self.VISITED:
                print('already visited ' + trial_name)
                fitnesses.append([candidate, self.VISITED[trial_name]])
            else:
                fitnesses.append([candidate, fitness(candidate)])
        # print(f'fitnesses: {fitnesses}')
        fitnesses.sort(key=lambda x: x[1], reverse=True) # high to low fitness
        # print(f'sorted fitnesses: {fitnesses}')
        selections = fitnesses[:p] 
        selections = [x[0] for x in selections] # len(selections) = p
        return selections

    
    def crossover(self, population) -> list:
        print("crossover")
        random.shuffle(population)
        crossover_pairs = [(population[i], population[i+1]) for i in range(0, len(population), 2)]
        crossovers = []  # len(crossovers) = n
        for pair in crossover_pairs:
            s1, s2 = pair
            cut_point = random.randint(1, len(s1) - 1) # split at a random point and join
            first_half = s1[:cut_point]
            second_half = s2.copy()
            # second_half.remove(parameter for parameter in first_half)
            for parameter in first_half:
                second_half.remove(parameter)
            crossover = first_half + second_half
            crossovers.append(crossover)

        return crossovers


    # TODO: include condition to check if a permutation has already been tested
    # TODO: why aren't we reaching crossover print statement?
    # TODO: why isn't the best fitness from initial population being chosen? ??????????


    def run(self, g=6.0e-8):
        
        # fitness = dummy_fitness # TODO: FOR DEBUGGING
        fitness = self.fitness

        # Generate n base permutations
        population = [random.sample(self.dataflow, len(self.dataflow)) for _ in range(self.n)]

        print("Initializing")
        # Initialize base fitness and goal fitness
        dfs_fitnesses = [[df, fitness(df)] for df in population]
        best_df, f = max(dfs_fitnesses, key=lambda x: x[1])

        for i in range(self.iter):
            print("\nITERATION: ", i)
            # Mutation
            mutations = self.mutation(population)
            # print(mutations)
            
            # Selection
            selections = self.selection(mutations, self.p, fitness, self.workload, self.pe_dims)
            # print(f'selections done running: {selections}')

            # TODO: we should also evaluate the best fitness here before crossing over or just add the eselections to crossovers

            # Crossover
            crossovers = self.crossover(selections)
            # print(f'crossovers done running: {crossovers}')

            crossovers.extend(selections)
            crossovers_fitnesses = []
            i = 0
            for crossover in crossovers:
                i += 1
                print(f'{i}/{len(crossovers)} candidate in crossover')
                trial_name = ''.join(crossover)
                if trial_name in self.VISITED:
                    print(f'already visited {trial_name}')
                    crossovers_fitnesses.append([crossover, self.VISITED[trial_name]])
                else: 
                    crossovers_fitnesses.append([crossover, fitness(crossover)])

            best_df_trial, f_trial = max(crossovers_fitnesses, key=lambda x: x[1])
            if f_trial > f:
                best_df, f = best_df_trial, f_trial
                print(f'new best trial: {best_df_trial} with fitness {f}')
            if f >= g:
                # goal fitness reached, break
                print('reached goal fitness, returning')
                break

        return best_df, f
        


ga = GeneticAlgorithm()
best_df, f = ga.run(g=1)

print(f'FINISHED RUNNING: FINAL FITNESS OF {f} ON PERMUTATION {best_df}\n TIMELOOP MAPPER CALLED {ga.mapper_call_count} NUMBER OF TIMES')

Initializing
[INFO] 2025-05-02 03:10:40,325 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.03 627200
['R', 'Q', 'M', 'S', 'C', 'N', 'P'] has a fitness of 2.8973064784699996e-08
[INFO] 2025-05-02 03:11:35,159 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.18 313600
['C', 'R', 'M', 'Q', 'P', 'S', 'N'] has a fitness of 5.7788610188548055e-08
[INFO] 2025-05-02 03:11:49,616 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


...

55.18 313600
['C', 'R', 'Q', 'M', 'S', 'N', 'P'] has a fitness of 5.7788610188548055e-08
3/15 candidate in crossover
already visited MQRSCPN
4/15 candidate in crossover
[INFO] 2025-05-02 04:31:47,062 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.18 313600
['C', 'R', 'N', 'M', 'Q', 'P', 'S'] has a fitness of 5.7788610188548055e-08
5/15 candidate in crossover
[INFO] 2025-05-02 04:32:00,922 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.03 627200
['S', 'R', 'M', 'Q', 'C', 'P', 'N'] has a fitness of 2.8973064784699996e-08
6/15 candidate in crossover
already visited RQMPCNS
7/15 candidate in crossover
already visited RQMSNCP
8/15 candidate in crossover
already visited CRMQNSP
9/15 candidate in crossover
already visited QRMSCNP
10/15 candidate in crossover
already visited MQRSCNP
11/15 candidate in crossover
already visited QPRCNSM
12/15 candidate in crossover
already visited CRSQPMN
13/15 candidate in crossover
already visited NMQCPSR
14/15 candidate in crossover
already visited SRMQPCN
15/15 candidate in crossover
already visited CRQMPSN
FINISHED RUNNING: FINAL FITNESS OF 6.058855235044807e-08 ON PERMUTATION ['M', 'P', 'C', 'S', 'Q', 'R', 'N']
 TIMELOOP MAPPER CALLED 121 NUMBER OF TIMES


In [61]:
for call in ga.mapper_calls:
    print(call)

['RQMSCNP', 2.8973064784699996e-08]
['CRMQPSN', 5.7788610188548055e-08]
['QRPCNSM', 2.8973064784699996e-08]
['PMCSNQR', 2.8973064784699996e-08]
['SMQCPNR', 2.8973064784699996e-08]
['CQMSRNP', 5.7788610188548055e-08]
['RQCSMNP', 2.8973064784699996e-08]
['SQMRCNP', 2.8973064784699996e-08]
['RSMQCNP', 2.8973064784699996e-08]
['SRMQPCN', 2.8973064784699996e-08]
['NRMQPSC', 5.1423568943784577e-08]
['PRQCNSM', 2.8973064784699996e-08]
['QNPCRSM', 2.8973064784699996e-08]
['QMPCNSR', 2.8973064784699996e-08]
['SRPCNQM', 2.8973064784699996e-08]
['PMRSNQC', 2.8973064784699996e-08]
['PNCSMQR', 2.8973064784699996e-08]
['RMCSNQP', 2.8973064784699996e-08]
['MPCSNQR', 6.058855235044807e-08]
['SMQCPRN', 2.8973064784699996e-08]
['SMQCRNP', 2.8973064784699996e-08]
['SQMCPNR', 2.8973064784699996e-08]
['MPCSQRN', 6.058855235044807e-08]
['RQSMPCN', 2.8973064784699996e-08]
['NRMQSCP', 5.1423568943784577e-08]
['PNRMQSC', 2.8973064784699996e-08]
['SCQMRNP', 2.8973064784699996e-08]
['RPMSCNQ', 2.8973064784699996

In [60]:
# saving this shit so we can save some time later
with open(f'fc1_2x8.yaml', 'w') as file:
            yaml.dump(ga.VISITED, file, default_flow_style=False)

## Workload: CONV1, PE: 1x16, 2x8, and 4x4

In [62]:
conv1_1x16 = GeneticAlgorithm(workload='layer_shapes/conv1.yaml', pe_dims={'pe_meshX': 1, 'pe_meshY': 16})
best_df, f = conv1_1x16.run(g=1)
print(f'Finished running conv1, 1x16: best fitness {f} on permutation {best_df}\n Timeloop mapper called {conv1_1x16.mapper_call_count} times')

Initializing
[INFO] 2025-05-02 05:13:41,586 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


68.57 245000
['C', 'N', 'P', 'Q', 'M', 'R', 'S'] has a fitness of 5.9525049629010125e-08
[INFO] 2025-05-02 05:14:51,859 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


61.61 280000
['M', 'P', 'S', 'Q', 'N', 'C', 'R'] has a fitness of 5.796832610661535e-08
[INFO] 2025-05-02 05:15:59,156 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


...

68.57 245000
['C', 'N', 'R', 'S', 'M', 'Q', 'P'] has a fitness of 5.9525049629010125e-08
2/15 candidate in crossover
[INFO] 2025-05-02 07:15:42,642 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.09 245000
['Q', 'C', 'P', 'S', 'M', 'N', 'R'] has a fitness of 7.40902641688369e-08
3/15 candidate in crossover
already visited QCPNSRM
4/15 candidate in crossover
already visited CSNMPRQ
5/15 candidate in crossover
[INFO] 2025-05-02 07:16:38,775 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.09 245000
['Q', 'C', 'M', 'S', 'R', 'P', 'N'] has a fitness of 7.40902641688369e-08
6/15 candidate in crossover
already visited CNRQMPS
7/15 candidate in crossover
already visited CSNMQPR
8/15 candidate in crossover
already visited QCPSMRN
9/15 candidate in crossover
already visited PMSQNCR
10/15 candidate in crossover
already visited QCPSNMR
11/15 candidate in crossover
already visited CNPQSRM
12/15 candidate in crossover
already visited CSNMPRQ
13/15 candidate in crossover
already visited QCRSNPM
14/15 candidate in crossover
already visited QCMSNRP
15/15 candidate in crossover
already visited CRPQMNS
Finished running conv1, 1x16: best fitness 7.40902641688369e-08 on permutation ['Q', 'C', 'P', 'S', 'N', 'R', 'M']
 Timeloop mapper called 121 times


In [63]:
for call in conv1_1x16.mapper_calls:
    print(call[0])

for call in conv1_1x16.mapper_calls:
    print(call[1])

with open('conv1_1x16.yaml', 'w') as file:
    yaml.dump(conv1_1x16.VISITED, file, default_flow_style=False)

CNPQMRS
MPSQNCR
CSNMRPQ
PMNQSCR
QCPSNRM
CNQPMRS
CRPQMNS
CNPQMSR
RNPQMCS
SPMQNCR
MPCQNSR
QPSMNCR
MPNQSCR
CMNSRPQ
QSNMRPC
CSNRMPQ
PMNQSRC
PMNQCSR
PMRQSCN
PMNCSQR
QCRSNPM
QCPMNRS
RNPQCMS
QCPMRSN
QCNPMSR
CNQRSPM
QPSMNRC
SNPQMRC
PNCQMRS
PMSQNCR
MPSCNQR
NPSQMCR
CSQMRPN
CPNMRSQ
NSCMRPQ
MSNCRPQ
PMCQSNR
CMNQSPR
PMNRSCQ
QCSPNRM
QCPSMRN
QCMSNRP
QCPSNMR
NSCPQMR
NCQPMRS
SNQCPMR
CNPQSRM
CQPNMRS
MPSQRCN
MRSQNCP
CSNQRPM
CNSMRPQ
CSPMRNQ
PQNMSCR
SMNQPCR
PCQSNRM
NCPSQRM
NPSQMRC
RNPQCSM
CNPSQRM
QCSPNMR
CNMQPRS
CNPMQRS
MPSRNCQ
CSNMQPR
QMNPSCR
NMPQSCR
QNPSCRM
QPCSNRM
NPSCQRM
QMCNPRS
NCPQMRS
MPRQNCS
RPSQNCM
PSNMRCQ
CQNMRPS
RSNMCPQ
CQPSNRM
QMPSNRC
QNPSMRC
RCPQMNS
QMPSNCR
CMPQNRS
MNPQCRS
MNSQPCR
MPSQCNR
QRPSNCM
RCPSNQM
QSNPMRC
CQPSMRN
NMPCQRS
CQSNMRP
RPQSNCM
CPSQNMR
MPQSNCR
QSPCNRM
MCPSNRQ
CPSQRNM
PQCSNRM
CQMPNRS
CMQSPNR
PCNQSMR
QCPNSRM
CMNQPRS
CQPNSRM
QCMPSNR
CPNQMRS
PRNQSCM
SCPQNRM
QCNSPRM
QCNSMRP
CNQMPRS
CPNQSMR
CPQNSRM
CNRQMPS
CSNMPRQ
CRNMSPQ
PSNQMCR
CNRSMQP
QCPSMNR
QCMSRPN
5.9525049629010125e-08
5.7968326

In [64]:
conv1_2x8 = GeneticAlgorithm(workload='layer_shapes/conv1.yaml', pe_dims={'pe_meshX': 2, 'pe_meshY': 8})
best_df, f = conv1_2x8.run(g=1)
print(f'Finished running conv1, 2x8: best fitness {f} on permutation {best_df}\n Timeloop mapper called {conv1_2x8.mapper_call_count} times')

Initializing
[INFO] 2025-05-02 07:17:36,243 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


68.56 245000
['P', 'S', 'Q', 'C', 'M', 'R', 'N'] has a fitness of 5.953373181244493e-08
[INFO] 2025-05-02 07:18:31,091 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


68.56 245000
['P', 'R', 'S', 'C', 'M', 'Q', 'N'] has a fitness of 5.953373181244493e-08
[INFO] 2025-05-02 07:19:24,917 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


...

57.57 490000
['C', 'S', 'M', 'R', 'Q', 'N', 'P'] has a fitness of 3.544930218048658e-08
20/20 candidate
already visited CSPQRNM
crossover
1/15 candidate in crossover
already visited PRNCMQS
2/15 candidate in crossover
[INFO] 2025-05-02 09:07:01,193 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.09 245000
['N', 'R', 'P', 'S', 'Q', 'C', 'M'] has a fitness of 7.40902641688369e-08
3/15 candidate in crossover
[INFO] 2025-05-02 09:08:02,197 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


68.56 245000
['Q', 'S', 'P', 'R', 'M', 'C', 'N'] has a fitness of 5.953373181244493e-08
4/15 candidate in crossover
already visited NPCRQMS
5/15 candidate in crossover
already visited PSQRMCN
6/15 candidate in crossover
already visited PRNCMQS
7/15 candidate in crossover
already visited NPCQSMR
8/15 candidate in crossover
already visited NRSCMQP
9/15 candidate in crossover
already visited PSQCRMN
10/15 candidate in crossover
already visited QSPCMRN
11/15 candidate in crossover
already visited PSQRMCN
12/15 candidate in crossover
already visited NSCPQMR
13/15 candidate in crossover
already visited NPCRQMS
14/15 candidate in crossover
already visited PSQRMCN
15/15 candidate in crossover
already visited PRMCSQN
Finished running conv1, 2x8: best fitness 7.40902641688369e-08 on permutation ['N', 'P', 'C', 'S', 'Q', 'M', 'R']
 Timeloop mapper called 116 times


In [65]:
for call in conv1_2x8.mapper_calls:
    print(call[0])

for call in conv1_2x8.mapper_calls:
    print(call[1])

with open('conv1_2x8.yaml', 'w') as file:
    yaml.dump(conv1_2x8.VISITED, file, default_flow_style=False)

PSQCMRN
PRSCMQN
NPCSQMR
RNCQPMS
CSMQRNP
PCQSMRN
QSPCMRN
PMQCSRN
PSRCMQN
PRMCSQN
PNSCMQR
PCSRMQN
NQCSPMR
RPCSQMN
NPCRQMS
RNQCPMS
MNCQPRS
RNCPQMS
MSCQRNP
QSMCRNP
SCMQRNP
PNSCMRQ
NQCSPRM
RSQCMPN
PSQRMCN
PSQNMRC
CSQPMRN
PRSCQMN
SRPCMQN
QRSCMPN
NRCSQMP
NPQSCMR
SPCNQMR
NPRSQMC
QNCRPMS
RQCNPMS
RNCQPSM
CSMQPNR
PSMQRNC
CSMQRPN
QNCRSMP
PRSQNMC
QNCRPSM
PSNQCMR
PSQMCRN
PSMCQRN
PNQCMRS
PRSCMNQ
CPNSQMR
QPCSNMR
PNCSQMR
RNCQMPS
CNRQPMS
CQMSRNP
CSMQNRP
QPNCMRS
PSNRQMC
PRSNMQC
PRSQMCN
NPMSQCR
NPCSRMQ
RNCQSMP
PNCQRMS
RSMQCNP
CMSQRNP
CSMPRNQ
QSPRNMC
PSQCMNR
MRSCPQN
PMSCRQN
NPCSQRM
MPCSQNR
RMCQPNS
RNMQPCS
RNSQPMC
CPMQRNS
NPCSMQR
NQPSRCM
PSNCMRQ
PSQCNRM
PQSCMRN
NMCSQPR
RSCQPMN
SNCQPMR
NSMQRCP
PRSNCMQ
PCSQRMN
PRQCMSN
SPQCMRN
NSQCMRP
NPCMQSR
RCNQPMS
CRMQSNP
QSMCPRN
NPCMQRS
PRSCNQM
NCPSQMR
RNCSPMQ
CSNQRMP
CSQMRNP
PRNCSQM
MNCQPSR
RNPQCMS
CSPQRNM
PRQSCNM
PSRCQNM
PRSCQNM
PSQCRMN
PRNCMQS
NRSCMQP
NSCPQMR
NPCQSMR
RPCQNMS
CSMRQNP
NRPSQCM
QSPRMCN
5.953373181244493e-08
5.953373181244493e-08
7.40902641688369e-08
3.54493